In [ ]:
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist, FieldError

In [ ]:
file = "legacy_data/20180830_Personen.xlsx"

In [ ]:
df = pd.read_excel(file).fillna('False')

In [ ]:
for i, row in df.iterrows():
    if row['Name'] != 'False':
        item, _ = Person.objects.get_or_create(written_name = row['Name'].strip())
        if row['Nachname'] != 'False':
            item.name = row['Nachname']
        if row['Vorname'] != 'False':
            item.forename = row['Vorname']
        if row['Titel'] != 'False':
            item.acad_title = row['Titel']
        if row['Alias'] != 'False':
            item.alt_names = row['Alias']
        if row['Funktionen'] != 'False':
            item.funktion = row['Funktionen']
        if row[' Mitgliedschaften'] != 'False':
            for x in row[' Mitgliedschaften'].split(';'):
                try:
                    pl = Institution.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.belongs_to_institution.add(pl)
        if row['Geburtsdatum'] != 'False':
            try:
                item.date_of_birth = pd.to_datetime(row['Geburtsdatum'])
            except Exception as e:
                print(e)
        if row['Sterbedatum'] != 'False':
            try:
                item.date_of_death = pd.to_datetime(row['Sterbedatum'])
            except Exception as e:
                print(e)
        if row['Geburtsort'] != 'False':
            pl = None
            try:
                pl = Place.objects.get(name=row['Geburtsort'])
            except Exception as e:
                print(e, row['Geburtsort'])
                pl = None
            if pl:
                item.place_of_birth = pl
        if row['Sterbeort'] != 'False':
            pl = None
            try:
                pl = Place.objects.get(name=row['Sterbeort'])
            except Exception as e:
                print(e, row['Sterbeort'])
                pl = None
            if pl:
                item.place_of_death = pl
        if row['Quelle'] != 'False':
            item.quelle = row['Quelle']
        if row['Weitere biographische Anmerkungen zur Person'] != 'False':
            item.biography = row['Weitere biographische Anmerkungen zur Person']
        item.save()